In [1]:
#r "./bin/Debug/net7.0/studies.dll"
#r "./bin/Debug/net7.0/core.fs.dll"
#r "nuget:FSharp.Data"
#r "nuget: Plotly.NET.Interactive"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed Packages FSharp.Data, 6.3.0 Plotly.NET.Interactive, 4.2.1

Loading extensions from `C:\Users\laimi\.nuget\packages\plotly.net.interactive\4.2.1\lib\netstandard2.1\Plotly.NET.Interactive.dll`

In [2]:
open FSharp.Data
open Plotly.NET

let trade_csv = "d:\\studies\\03_export_date_ticker_screenerid_gap_outcomes.csv"

let csv = studies.GapStudy.parseTradeOutcomes trade_csv

In [3]:
let outputSummary includeChart (summary:studies.Types.TradeSummary) = 

    printfn "%s:\n %i trades, %.2f%% win pct, %.2f%% avg gain, %.2f%% avg loss, %.2f avg gain/loss, %.2f EV" 
        summary.StrategyName summary.Total (summary.WinPct * 100m) (summary.AvgWin) (summary.AvgLoss) summary.AvgGainLoss summary.EV

    match includeChart with
    | true ->
        let chart = 
            Chart.Histogram(
                X = summary.Gains,
                NBinsX = 40,
                Name = "Gain Distribution"
            )

        chart.Display() |> ignore
    | false -> ()

In [8]:
let filterNamePairs = [
        ("All", fun (o:studies.Types.TradeOutcomeOutput.Row) -> true)
        ("New Highs", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.Screenerid = 28)
        ("Top Gainers", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.Screenerid = 29)
        ("Gap ups", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.HasGapUp)
        ("Gap ups - new highs", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.HasGapUp && o.Screenerid = 28)
        ("Gap ups - top gainers", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.HasGapUp && o.Screenerid = 29)
    ]

let createTradeSummaries outcomes =
    
    let tradesGroupedByStrategy =
        outcomes
        |> Seq.groupBy (fun (t:studies.Types.TradeOutcomeOutput.Row) -> t.Strategy) // all strategies, no filters

    filterNamePairs
    |> Seq.collect (
        fun (filterName,filter) ->

            tradesGroupedByStrategy
            |> Seq.map (
                fun (strategy, trades) ->
                    let filteredTrades = trades |> Seq.filter filter
                    let strategy = filteredTrades |> Seq.head |> fun x -> x.Strategy
                    let name = $"{filterName}, {strategy}"
                    studies.Types.TradeSummary.create name filteredTrades
            )
    )
    

let describeOutcomes includeChart tradeSummaries =
    
    tradeSummaries
    |> Seq.iter (
        fun summary -> outputSummary includeChart summary
    )
    
let highlightStrategies numberOfTopRecords tradeSummaries =
    
    let sortFunctions = [
        ("EV", fun (t:studies.Types.TradeSummary) -> t.EV)
        ("Win %", fun (t:studies.Types.TradeSummary) -> t.WinPct)
        ("Avg Gain", fun (t:studies.Types.TradeSummary) -> t.AvgGainLoss)
    ]

    sortFunctions
    |> Seq.iter (fun (sortLabel, sortFunction) ->
        printfn $"Top {numberOfTopRecords} by: {sortLabel}"
        tradeSummaries
        |> Seq.sortByDescending sortFunction
        |> Seq.take numberOfTopRecords
        |> Seq.iter (fun summary ->
            printfn $"{summary.StrategyName}: {sortFunction summary}"
        )
        printfn ""
        printfn ""
    )

In [9]:
let studiesDirectory = "d:\\studies"
let signalPath = "d:\\studies\\02_export_date_ticker_screenerid_gap.csv"
let priceFunc = studies.DataHelpers.getPricesFromCsv studiesDirectory

let runStrategies strategies = async {
    let! signalsWithPrices = studies.Trading.prepareSignalsForTradeSimulations signalPath priceFunc
    
    return! studies.Trading.runTrades signalsWithPrices strategies
}

In [10]:
let strategies = 
        [
            studies.TradingStrategies.buyAndHoldStrategy (Some 5)
            studies.TradingStrategies.buyAndHoldStrategy (Some 10)
            studies.TradingStrategies.buyAndHoldStrategy (Some 30)
            studies.TradingStrategies.buyAndHoldStrategy (Some 60)
            studies.TradingStrategies.buyAndHoldStrategy (Some 90)
            studies.TradingStrategies.buyAndHoldStrategy None
            studies.TradingStrategies.buyAndHoldStrategyWithStopLoss false (Some 5) 5m
            studies.TradingStrategies.buyAndHoldStrategyWithStopLoss false (Some 10) 5m
            studies.TradingStrategies.buyAndHoldStrategyWithStopLoss false (Some 30) 5m
            studies.TradingStrategies.buyAndHoldStrategyWithStopLoss false (Some 60) 5m
            studies.TradingStrategies.buyAndHoldStrategyWithStopLoss false (Some 90) 5m
            studies.TradingStrategies.buyAndHoldStrategyWithStopLoss false None 5m
        ]

let outcomes = 
    strategies
    |> runStrategies
    |> Async.RunSynchronously

printfn "Finished running trades, generated:"
printfn $"{outcomes |> Seq.length} trade outcomes"

Records: 23520, dates: 423, tickers: 3164, screenerIds: 2
Minimum date: 3/22/2022 12:00:00 AM
Maximum date: 11/13/2023 12:00:00 AM

Ensured that data has prices
Records: 22848, dates: 412, tickers: 3153, screenerIds: 2
Minimum date: 3/22/2022 12:00:00 AM
Maximum date: 11/13/2023 12:00:00 AM

Executing trades...
Finished running trades, generated:
274176 trade outcomes


In [11]:
let outcomeDescription (x:studies.Types.TradeOutcomeOutput.Row) =
    printfn $"{x.Ticker} {x.Strategy}: {x.Opened.ToShortDateString()} @ {x.OpenPrice}, {x.Closed.ToShortDateString()} @ {x.ClosePrice}: profit of {x.PercentGain}"
    
let random = System.Random()

outcomes
|> Seq.sortBy (fun x -> random.NextInt64())
|> Seq.take 10
|> Seq.iter outcomeDescription


FLNC Buy and Hold 5 bars: 3/31/2023 @ 19.38, 4/10/2023 @ 19.81: profit of 2.2187822497420020639834881300
PLTR Buy and Hold 60 bars with Stop Loss of 5.00%: 5/24/2023 @ 12.05, 8/21/2023 @ 14.5: profit of 20.331950207468879668049792530
RDFN Buy and Hold 90 bars with Stop Loss of 5.00%: 5/13/2022 @ 11.07, 5/16/2022 @ 10.35: profit of -6.5040650406504065040650406500
TAK Buy and Hold with Stop Loss of 5.00%: 12/15/2022 @ 15.3, 10/19/2023 @ 14.53: profit of -5.0326797385620915032679738600
JWN Buy and Hold 10 bars with Stop Loss of 5.00%: 6/2/2023 @ 16.36, 6/16/2023 @ 19.06: profit of 16.503667481662591687041564790
LPX Buy and Hold 60 bars: 5/5/2022 @ 75.67, 8/2/2022 @ 62.24: profit of -17.748116823047442843927580280
IDYA Buy and Hold 10 bars with Stop Loss of 5.00%: 5/10/2023 @ 22.62, 5/11/2023 @ 21.44: profit of -5.216622458001768346595932800
GFF Buy and Hold 10 bars: 8/2/2023 @ 43.49, 8/16/2023 @ 42.34: profit of -2.6442860427684525178201885500
PAY Buy and Hold 90 bars with Stop Loss of 5.

In [12]:
// inspect what's max gain 
outcomes
|> Seq.filter (fun x -> x.Strategy.Contains("with Stop Loss"))
|> Seq.maxBy (fun x -> x.PercentGain)
|> outcomeDescription


SMCI Buy and Hold with Stop Loss of 5.00%: 7/22/2022 @ 50.92, 11/14/2023 @ 282.25: profit of 454.30086410054988216810683425


In [13]:
// investigating what happened with the trade and why it went as it went
let signalsWithPrices = studies.Trading.prepareSignalsForTradeSimulations signalPath priceFunc |> Async.RunSynchronously

let tickerOfInterest = "MLTX"

let signal, prices = 
    signalsWithPrices
    |> Seq.filter (fun (signal, prices) -> signal.Ticker = tickerOfInterest)
    |> Seq.head
    
let strategy = studies.TradingStrategies.buyAndHoldStrategyWithStopLoss true (Some 5) 5m

(signal, prices) |> strategy |> outcomeDescription

Records: 23520, dates: 423, tickers: 3164, screenerIds: 2
Minimum date: 3/22/2022 12:00:00 AM
Maximum date: 11/13/2023 12:00:00 AM

Ensured that data has prices
Records: 22848, dates: 412, tickers: 3153, screenerIds: 2
Minimum date: 3/22/2022 12:00:00 AM
Maximum date: 11/13/2023 12:00:00 AM

Open day for MLTX on 12/28/2022 12:00:00 AM is 12/29/2022 12:00:00 AM +00:00 with open day index of 183
Close day for MLTX on 12/28/2022 12:00:00 AM is based on Some 5 number of bars to hold and is 1/6/2023 12:00:00 AM +00:00 with close day index of 188
Close bar for MLTX on 12/28/2022 12:00:00 AM is 1/6/2023 12:00:00 AM +00:00 because close day reached
MLTX Buy and Hold 5 bars with Stop Loss of 5.00%: 12/29/2022 @ 10.22, 1/6/2023 @ 12.52: profit of 22.504892367906066536203522500


In [15]:
let tradeSummaries = outcomes |> createTradeSummaries

tradeSummaries |> describeOutcomes false

All, Buy and Hold 5 bars:
 22848 trades, 50.28% win pct, 7.40% avg gain, -6.87% avg loss, 1.08 avg gain/loss, 0.30 EV
All, Buy and Hold 10 bars:
 22848 trades, 49.16% win pct, 9.15% avg gain, -9.07% avg loss, 1.01 avg gain/loss, -0.12 EV
All, Buy and Hold 30 bars:
 22848 trades, 45.54% win pct, 14.08% avg gain, -14.67% avg loss, 0.96 avg gain/loss, -1.58 EV
All, Buy and Hold 60 bars:
 22848 trades, 44.82% win pct, 18.24% avg gain, -18.96% avg loss, 0.96 avg gain/loss, -2.29 EV
All, Buy and Hold 90 bars:
 22848 trades, 44.18% win pct, 20.46% avg gain, -21.37% avg loss, 0.96 avg gain/loss, -2.89 EV
All, Buy and Hold:
 22848 trades, 43.08% win pct, 29.77% avg gain, -30.46% avg loss, 0.98 avg gain/loss, -4.52 EV
All, Buy and Hold 5 bars with Stop Loss of 5.00%:
 22848 trades, 47.19% win pct, 7.44% avg gain, -5.75% avg loss, 1.29 avg gain/loss, 0.47 EV
All, Buy and Hold 10 bars with Stop Loss of 5.00%:
 22848 trades, 42.35% win pct, 9.32% avg gain, -6.59% avg loss, 1.41 avg gain/loss, 0.15 

 1737 trades, 20.21% win pct, 26.98% avg gain, -6.82% avg loss, 3.96 avg gain/loss, 0.01 EV
Gap ups - top gainers, Buy and Hold 5 bars:
 4710 trades, 51.55% win pct, 7.54% avg gain, -6.84% avg loss, 1.10 avg gain/loss, 0.57 EV
Gap ups - top gainers, Buy and Hold 10 bars:
 4710 trades, 51.32% win pct, 9.25% avg gain, -8.78% avg loss, 1.05 avg gain/loss, 0.47 EV
Gap ups - top gainers, Buy and Hold 30 bars:
 4710 trades, 48.07% win pct, 14.06% avg gain, -14.34% avg loss, 0.98 avg gain/loss, -0.69 EV
Gap ups - top gainers, Buy and Hold 60 bars:
 4710 trades, 49.13% win pct, 18.65% avg gain, -18.34% avg loss, 1.02 avg gain/loss, -0.17 EV
Gap ups - top gainers, Buy and Hold 90 bars:
 4710 trades, 46.56% win pct, 20.14% avg gain, -20.24% avg loss, 1.00 avg gain/loss, -1.44 EV
Gap ups - top gainers, Buy and Hold:
 4710 trades, 44.25% win pct, 31.64% avg gain, -29.24% avg loss, 1.08 avg gain/loss, -2.31 EV
Gap ups - top gainers, Buy and Hold 5 bars with Stop Loss of 5.00%:
 4710 trades, 48.56% 

In [ ]:
tradeSummaries |> highlightStrategies 10

Top 10 by: EV
